### Cluster Analysis

This notebook details the process of clustering the different financial institutions in the West African Economic and Moetary Union (UEMOA). Initially I thought of clustering the menber countries themselves but given they are just 8 and posses quite a diverse economic landscape, I thought it would be more effectov